<a href="https://colab.research.google.com/github/ProfKaromo/Data_Science_Codes/blob/main/Text_to_Speech_Kerras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from tqdm import tqdm

In [3]:
# Subsample
labels = [
    'down', 'left', 'up',
]
train_audio_path = '/content/drive/MyDrive/audio'

all_wave = []
all_label = []
for label in tqdm(labels):
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        samples, sample_rate = librosa.load(train_audio_path + '/' + label + '/' + wav, sr = 16000)
        samples = librosa.resample(samples, sample_rate, 8000)
        if(len(samples)== 8000) : 
            all_wave.append(samples)
            all_label.append(label)

100%|██████████| 3/3 [00:13<00:00,  4.49s/it]


In [4]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

label_enconder = LabelEncoder()
y = label_enconder.fit_transform(all_label)
classes = list(label_enconder.classes_)
classes
y = np_utils.to_categorical(y, num_classes=len(labels))

In [5]:
all_wave = np.array(all_wave).reshape(-1,8000,1)

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(np.array(all_wave),np.array(y),stratify=y,test_size = 0.2,random_state=777,shuffle=True)

In [7]:
from keras.layers import Bidirectional, BatchNormalization, CuDNNGRU, TimeDistributed

In [8]:
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
K.clear_session()

inputs = Input(shape=(8000,1))
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(inputs)

#First Conv1D layer
x = Conv1D(8,13, padding='valid', activation='relu', strides=1)(x)
x = MaxPooling1D(3)(x)
x = Dropout(0.3)(x)

#Second Conv1D layer
x = Conv1D(16, 11, padding='valid', activation='relu', strides=1)(x)
x = MaxPooling1D(3)(x)
x = Dropout(0.3)(x)

#Third Conv1D layer
x = Conv1D(32, 9, padding='valid', activation='relu', strides=1)(x)
x = MaxPooling1D(3)(x)
x = Dropout(0.3)(x)

x = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(x)
x = Bidirectional(CuDNNGRU(128, return_sequences=True), merge_mode='sum')(x)
x = Bidirectional(CuDNNGRU(128, return_sequences=True), merge_mode='sum')(x)
x = Bidirectional(CuDNNGRU(128, return_sequences=False), merge_mode='sum')(x)

x = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(x)

#Flatten layer
# x = Flatten()(x)

#Dense Layer 1
x = Dense(256, activation='relu')(x)
outputs = Dense(len(labels), activation="softmax")(x)

model = Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8000, 1)]         0         
                                                                 
 batch_normalization (BatchN  (None, 8000, 1)          4         
 ormalization)                                                   
                                                                 
 conv1d (Conv1D)             (None, 7988, 8)           112       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2662, 8)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 2662, 8)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 2652, 16)          1424  

In [9]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [10]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', 
                           verbose=1, patience=10, min_delta=0.0001)

checkpoint = ModelCheckpoint('speech2text_model.hdf5', monitor='val_acc', 
                             verbose=1, save_best_only=True, mode='max')

In [11]:
hist = model.fit(
    x=x_train, 
    y=y_train,
    epochs=3, 
    callbacks=[early_stop, checkpoint], 
    batch_size=32, 
    validation_data=(x_valid,y_valid)
)

Epoch 1/3
1/1 [==============================] - ETA: 0s - loss: 1.1049 - accuracy: 0.3125

1/1 [==============================] - 12s 12s/step - loss: 1.1049 - accuracy: 0.3125 - val_loss: 1.0968 - val_accuracy: 0.5000
Epoch 2/3
1/1 [==============================] - ETA: 0s - loss: 1.0527 - accuracy: 0.5625

1/1 [==============================] - 0s 124ms/step - loss: 1.0527 - accuracy: 0.5625 - val_loss: 1.0935 - val_accuracy: 0.5000
Epoch 3/3
1/1 [==============================] - ETA: 0s - loss: 1.0287 - accuracy: 0.5000

1/1 [==============================] - 0s 117ms/step - loss: 1.0287 - accuracy: 0.5000 - val_loss: 1.0876 - val_accuracy: 0.5000


In [12]:
model.save('speech2text_model.hdf5')

In [13]:
from keras.models import load_model
model = load_model('speech2text_model.hdf5')

In [17]:
def s2t_predict(audio, shape_num=8000):
    prob = model.predict(audio.reshape(1,-1,1))   #audio.reshape((1, -1, 1))
    index = np.argmax(prob[0])
    return classes[index]

In [19]:
audio, audio_rate = librosa.load('0a2b400e_nohash_2.wav')
audio_sample = librosa.resample(audio, audio_rate, 8000)
ipd.Audio(audio_sample,rate=8000)
audio_sample

array([-4.1024570e-05, -9.9386401e-05, -6.5733962e-05, ...,
        1.2794257e-05,  1.4792475e-05,  2.3976547e-05], dtype=float32)

In [20]:
print("Predicted Text:", s2t_predict(audio_sample))

1/1 [==============================] - 1s 683ms/step
Predicted Text: left
